In [1]:
# librerías 
import re
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

In [2]:
prime_df = pd.read_csv("./amazon_prime_titles.csv", sep=",")
prime_df["streaming_service"] = "Amazon Prime"
prime_df = prime_df.rename(columns={"listed_in": "genre"})
print(prime_df.info())
display(prime_df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9668 entries, 0 to 9667
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   show_id            9668 non-null   object
 1   type               9668 non-null   object
 2   title              9668 non-null   object
 3   director           7586 non-null   object
 4   cast               8435 non-null   object
 5   country            672 non-null    object
 6   date_added         155 non-null    object
 7   release_year       9668 non-null   int64 
 8   rating             9331 non-null   object
 9   duration           9668 non-null   object
 10  genre              9668 non-null   object
 11  description        9668 non-null   object
 12  streaming_service  9668 non-null   object
dtypes: int64(1), object(12)
memory usage: 982.0+ KB
None


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,genre,description,streaming_service
0,s1,Movie,The Grand Seduction,Don McKellar,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",Canada,"March 30, 2021",2014,NaN,113 min,"Comedy, Drama",A small fishing village must procure a local d...,Amazon Prime
1,s2,Movie,Take Care Good Night,Girish Joshi,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",India,"March 30, 2021",2018,13+,110 min,"Drama, International",A Metro Family decides to fight a Cyber Crimin...,Amazon Prime
2,s3,Movie,Secrets of Deception,Josh Webber,"Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...",United States,"March 30, 2021",2017,NaN,74 min,"Action, Drama, Suspense",After a man discovers his wife is cheating on ...,Amazon Prime
3,s4,Movie,Pink: Staying True,Sonia Anderson,"Interviews with: Pink, Adele, Beyoncé, Britney...",United States,"March 30, 2021",2014,NaN,69 min,Documentary,"Pink breaks the mold once again, bringing her ...",Amazon Prime
4,s5,Movie,Monster Maker,Giles Foster,"Harry Dean Stanton, Kieran O'Brien, George Cos...",United Kingdom,"March 30, 2021",1989,NaN,45 min,"Drama, Fantasy",Teenage Matt Banting wants to work with a famo...,Amazon Prime


In [3]:
# Convertimos las fechas a tipo fecha
prime_df["date_added"] = pd.to_datetime(prime_df["date_added"])

# Extraemos el Año, Mes, Día de la fecha que se añadio a la plataforma
prime_df["year_added"] = prime_df["date_added"].dt.strftime('%Y')
prime_df["month_added"] = prime_df["date_added"].dt.strftime('%m')
prime_df["day_added"] = prime_df["date_added"].dt.strftime('%d')

# Convertimos a lista los países y géneros
prime_df["country"] = prime_df["country"].str.split(", ")
prime_df["genre"] = prime_df["genre"].str.split(", ")

# Dividimos la columna duration en dos:
# ['duration'] contiene el núm. de minutos o temporadas.
# ['duration_type'] contiene el el tipo de duración. de min (minutos) o Seasons (temporadas).
prime_df[['duration','duration_type']] = prime_df["duration"].str.split(" ",expand=True,)

display(prime_df.head())

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,genre,description,streaming_service,year_added,month_added,day_added,duration_type
0,s1,Movie,The Grand Seduction,Don McKellar,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",[Canada],2021-03-30,2014,NaN,113,"[Comedy, Drama]",A small fishing village must procure a local d...,Amazon Prime,2021,03,30,min
1,s2,Movie,Take Care Good Night,Girish Joshi,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",[India],2021-03-30,2018,13+,110,"[Drama, International]",A Metro Family decides to fight a Cyber Crimin...,Amazon Prime,2021,03,30,min
2,s3,Movie,Secrets of Deception,Josh Webber,"Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...",[United States],2021-03-30,2017,NaN,74,"[Action, Drama, Suspense]",After a man discovers his wife is cheating on ...,Amazon Prime,2021,03,30,min
3,s4,Movie,Pink: Staying True,Sonia Anderson,"Interviews with: Pink, Adele, Beyoncé, Britney...",[United States],2021-03-30,2014,NaN,69,[Documentary],"Pink breaks the mold once again, bringing her ...",Amazon Prime,2021,03,30,min
4,s5,Movie,Monster Maker,Giles Foster,"Harry Dean Stanton, Kieran O'Brien, George Cos...",[United Kingdom],2021-03-30,1989,NaN,45,"[Drama, Fantasy]",Teenage Matt Banting wants to work with a famo...,Amazon Prime,2021,03,30,min


In [4]:
# Dividimos en dos dataframes
prime_movies_df = prime_df[prime_df['type'] == 'Movie'].copy()
print(f"Movies: {prime_movies_df.shape}")
prime_series_df = prime_df[prime_df['type'] == 'TV Show'].copy()
print(f"Series: {prime_series_df.shape}")

Movies: (7814, 17)
Series: (1854, 17)


## Movies
---
Limpieza de datos de películas (Rating: https://www.primevideo.com/help/ref=atv_hp_nd_cnt?nodeId=GFGQU3WYEG6FSJFJ )

In [5]:
# movie_rating = ['G', 'PG', 'PG-13', 'R', 'NC-17']
prime_movies_df['rating'].unique()
# APTA = ALL_AGES, ALL, G
apta_list = ['ALL_AGES', 'ALL', 'G']

# 7 (7+) = 7+, PG
siete_list = ['7+', 'PG']

# 12 (13+) = 13+
doce_list = ['13+']
# 16 (16+) = 16+, AGES_16_, 16
mayores_16_list = ['16+', 'AGES_16_', '16']
# 18 (18+) = 18+, AGES_18_, NC-17, R
mayores_18_list = ['18+', 'AGES_18_', 'NC-17', 'R']
# Sin calificación = NOT_RATE, UNRATED, NR, nan
sc_list = ['NOT_RATE', 'UNRATED', 'NR', 'nan']

# Condiciones para renombrar la clasificación por edades de las películas
conditions = [prime_movies_df['rating'].isin(apta_list),
              prime_movies_df['rating'].isin(siete_list),
              prime_movies_df['rating'].isin(doce_list),
              prime_movies_df['rating'].isin(mayores_16_list),
              prime_movies_df['rating'].isin(mayores_18_list),
              prime_movies_df['rating'].isin(sc_list),
             ]
# Resultado de la clasificación anterior por edades de las películas
results = ['APTA', 
           '7', 
           '12', 
           '16', 
           '18', 
           'Sin calificación'
          ]

# Renombramos la clasificación por edades
prime_movies_df['rating'] = np.select(conditions,results, default=np.nan)

In [6]:
prime_movies_df.tail(3)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,genre,description,streaming_service,year_added,month_added,day_added,duration_type
9663,s9664,Movie,Pride Of The Bowery,Joseph H. Lewis,"Leo Gorcey, Bobby Jordan",NaN,NaT,1940,7,60,[Comedy],New York City street principles get an East Si...,Amazon Prime,NaN,NaN,NaN,min
9665,s9666,Movie,Outpost,Steve Barker,"Ray Stevenson, Julian Wadham, Richard Brake, M...",NaN,NaT,2008,18,90,[Action],"In war-torn Eastern Europe, a world-weary grou...",Amazon Prime,NaN,NaN,NaN,min
9667,s9668,Movie,Harry Brown,Daniel Barber,"Michael Caine, Emily Mortimer, Joseph Gilgun, ...",NaN,NaT,2010,18,103,"[Action, Drama, Suspense]","Harry Brown, starring two-time Academy Award w...",Amazon Prime,NaN,NaN,NaN,min


### Juntamos los datos de ImBD

In [7]:
imbd_df = pd.read_csv("./ImBD_data/IMDb movies.csv", sep=",")
print(imbd_df.info())
display(imbd_df.head(2))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85855 entries, 0 to 85854
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   imdb_title_id          85855 non-null  object 
 1   title                  85855 non-null  object 
 2   original_title         85855 non-null  object 
 3   year                   85855 non-null  object 
 4   date_published         85855 non-null  object 
 5   genre                  85855 non-null  object 
 6   duration               85855 non-null  int64  
 7   country                85791 non-null  object 
 8   language               85022 non-null  object 
 9   director               85768 non-null  object 
 10  writer                 84283 non-null  object 
 11  production_company     81400 non-null  object 
 12  actors                 85786 non-null  object 
 13  description            83740 non-null  object 
 14  avg_vote               85855 non-null  float64
 15  vo

/opt/anaconda3/envs/PRVD_Project/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,...,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics
0,tt0000009,Miss Jerry,Miss Jerry,1894,1894-10-09,Romance,45,USA,None,Alexander Black,...,"Blanche Bayliss, William Courtenay, Chauncey D...",The adventures of a female reporter in the 1890s.,5.9,154,NaN,NaN,NaN,NaN,1.0,2.0
1,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,1906-12-26,"Biography, Crime, Drama",70,Australia,None,Charles Tait,...,"Elizabeth Tait, John Tait, Norman Campbell, Be...",True story of notorious Australian outlaw Ned ...,6.1,589,$ 2250,NaN,NaN,NaN,7.0,7.0


In [21]:
name_title = prime_movies_df['title'].sample().values[0]
display(prime_movies_df[prime_movies_df['title']==name_title])
display(imbd_df[imbd_df['original_title']==name_title])

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,genre,description,streaming_service,year_added,month_added,day_added,duration_type
8600,s8601,Movie,Lonely Hearts,Todd Robinson,"James Gandolfini, John Travolta, Salma Hayek",NaN,NaT,2007,18,108,"[Drama, Suspense]","John Travolta, James Gandolfini, Salma Hayek, ...",Amazon Prime,NaN,NaN,NaN,min


,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,...,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics
20489,tt0084264,Lonely Hearts,Lonely Hearts,1982,1982-10-21,"Comedy, Drama, Romance",106,Australia,English,Paul Cox,...,"Wendy Hughes, Norman Kaye, Jon Finlayson, Juli...","In this offbeat comedy, an unlikely romance de...",6.8,325,NaN,$ 777087,$ 777087,NaN,8.0,8.0
26063,tt0102329,Annunci di morte,Lonely Hearts,1991,1993-10-16,"Drama, Romance, Thriller",109,USA,English,Andrew Lane,...,"Eric Roberts, Beverly D'Angelo, Joanna Cassidy...","Alma, a lonely woman, falls for the conman who...",4.8,282,NaN,NaN,NaN,NaN,6.0,2.0
46885,tt0441774,Lonely Hearts,Lonely Hearts,2006,2006-10-21,"Biography, Crime, Drama",108,"Germany, USA",English,Todd Robinson,...,"John Travolta, James Gandolfini, Jared Leto, S...","Based on the true story, two homicide detectiv...",6.4,19981,$ 18000000,$ 188565,$ 2520835,60.0,79.0,76.0


In [22]:
merged_df = pd.merge(prime_movies_df, 
                     imbd_df[['imdb_title_id', 'title', 'original_title', 'director', 'avg_vote', 'votes', 'production_company', 
                              'duration', 'year', 'genre', 'usa_gross_income', 'worlwide_gross_income']], 
                     left_on=['title','director'],
                     right_on=['original_title', 'director'],)
merged_df.shape

(1929, 28)

In [23]:
merged_df

,show_id,type,title_x,director,cast,country,date_added,release_year,rating,duration_x,...,title_y,original_title,avg_vote,votes,production_company,duration_y,year,genre_y,usa_gross_income,worlwide_gross_income
0,s1,Movie,The Grand Seduction,Don McKellar,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",[Canada],2021-03-30,2014,nan,113,...,The Grand Seduction,The Grand Seduction,7.0,15391,Max Films Productions,113,2013,Comedy,$ 3430018,$ 4292517
1,s2,Movie,Take Care Good Night,Girish Joshi,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",[India],2021-03-30,2018,12,110,...,Take Care Good Night,Take Care Good Night,7.4,149,Everest Entertainment,110,2018,"Crime, Drama, Family",NaN,NaN
2,s3,Movie,Secrets of Deception,Josh Webber,"Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...",[United States],2021-03-30,2017,nan,74,...,Secrets of Deception,Secrets of Deception,4.2,229,Webber Films,76,2017,"Action, Crime, Drama",NaN,NaN
3,s12,Movie,Take Care,Liz Tuccillo,"Leslie Bibb, Kevin Curtis, Nadia Dajani",[United States],2021-04-10,2014,nan,93,...,Take Care,Take Care,6.1,2873,NaN,94,2014,"Comedy, Drama, Romance",NaN,NaN
4,s16,Movie,Summer '03,Becca Gleason,"Joey King, Jack Kilmer, Andrea Savage, Paul Sc...",[United States],2021-06-03,2019,nan,96,...,Summer '03,Summer '03,5.4,2424,Big Cat Productions,95,2018,"Comedy, Drama, Romance",$ 11746,$ 11746
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1924,s9649,Movie,Judge Dredd,Danny Cannon,"Sylvester Stallone, Armand Assante, Rob Schneider",NaN,NaT,1995,18,96,...,Dredd - La legge sono io,Judge Dredd,5.6,108618,Hollywood Pictures,96,1995,"Action, Crime, Sci-Fi",$ 34693481,$ 113493481
1925,s9650,Movie,Jack,Francis Ford Coppola,"Robin Williams, Diane Lane, Brian Kerwin, Jenn...",NaN,NaT,1996,nan,113,...,Jack,Jack,5.8,55279,Hollywood Pictures,113,1996,"Comedy, Drama, Fantasy",$ 58620973,$ 58620973
1926,s9660,Movie,10 Things I Hate About You,Gil Junger,"Heath Ledger, Julia Stiles, Joseph Gordon-Levi...",NaN,NaT,1999,nan,97,...,10 cose che odio di te,10 Things I Hate About You,7.3,289989,Touchstone Pictures,97,1999,"Comedy, Drama, Romance",$ 38178166,$ 53485989
1927,s9666,Movie,Outpost,Steve Barker,"Ray Stevenson, Julian Wadham, Richard Brake, M...",NaN,NaT,2008,18,90,...,Outpost,Outpost,5.9,16001,Black Camel Pictures,90,2008,"Action, Horror, Sci-Fi",NaN,$ 463377


## Series
--- 
Limpieza de datos de series

In [85]:
tv_rating = ['TV-Y', 'TV-Y7', 'TV-G', 'TV-PG', 'TV-14', 'TV-MA']
prime_series_df['rating'].unique()

array(['ALL', '18+', 'TV-Y', 'TV-Y7', '16+', 'TV-PG', '13+', '7+',
       'TV-14', 'TV-NR', 'TV-G', 'TV-MA', nan, 'NR'], dtype=object)

In [86]:
# Obtener el número de temporadas
prime_series_df['duration'].unique()

array(['1 Season', '3 Seasons', '2 Seasons', '4 Seasons', '7 Seasons',
       '9 Seasons', '6 Seasons', '5 Seasons', '29 Seasons', '19 Seasons',
       '21 Seasons', '8 Seasons', '12 Seasons', '11 Seasons',
       '10 Seasons', '14 Seasons', '15 Seasons'], dtype=object)

In [87]:

prime_df.head(5)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,genre,description,streaming_service,year_added,month_added,day_added
0,s1,Movie,The Grand Seduction,Don McKellar,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",[Canada],2021-03-30,2014,NaN,113 min,"[Comedy, Drama]",A small fishing village must procure a local d...,Amazon Prime,2021,03,30
1,s2,Movie,Take Care Good Night,Girish Joshi,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",[India],2021-03-30,2018,13+,110 min,"[Drama, International]",A Metro Family decides to fight a Cyber Crimin...,Amazon Prime,2021,03,30
2,s3,Movie,Secrets of Deception,Josh Webber,"Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...",[United States],2021-03-30,2017,NaN,74 min,"[Action, Drama, Suspense]",After a man discovers his wife is cheating on ...,Amazon Prime,2021,03,30
3,s4,Movie,Pink: Staying True,Sonia Anderson,"Interviews with: Pink, Adele, Beyoncé, Britney...",[United States],2021-03-30,2014,NaN,69 min,[Documentary],"Pink breaks the mold once again, bringing her ...",Amazon Prime,2021,03,30
4,s5,Movie,Monster Maker,Giles Foster,"Harry Dean Stanton, Kieran O'Brien, George Cos...",[United Kingdom],2021-03-30,1989,NaN,45 min,"[Drama, Fantasy]",Teenage Matt Banting wants to work with a famo...,Amazon Prime,2021,03,30


In [25]:
# Almacenamos el total de países por película
#prime_df["num_countries"] = [len(x) for x in prime_df['country']]

# Almacenamos el total de géneros por película
prime_df["num_genders"] = [len(x) for x in prime_df['genre']]
prime_df

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,genre,description,streaming_service,year_added,month_added,day_added,duration_type,num_genders
0,s1,Movie,The Grand Seduction,Don McKellar,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",[Canada],2021-03-30,2014,NaN,113,"[Comedy, Drama]",A small fishing village must procure a local d...,Amazon Prime,2021,03,30,min,2
1,s2,Movie,Take Care Good Night,Girish Joshi,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",[India],2021-03-30,2018,13+,110,"[Drama, International]",A Metro Family decides to fight a Cyber Crimin...,Amazon Prime,2021,03,30,min,2
2,s3,Movie,Secrets of Deception,Josh Webber,"Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...",[United States],2021-03-30,2017,NaN,74,"[Action, Drama, Suspense]",After a man discovers his wife is cheating on ...,Amazon Prime,2021,03,30,min,3
3,s4,Movie,Pink: Staying True,Sonia Anderson,"Interviews with: Pink, Adele, Beyoncé, Britney...",[United States],2021-03-30,2014,NaN,69,[Documentary],"Pink breaks the mold once again, bringing her ...",Amazon Prime,2021,03,30,min,1
4,s5,Movie,Monster Maker,Giles Foster,"Harry Dean Stanton, Kieran O'Brien, George Cos...",[United Kingdom],2021-03-30,1989,NaN,45,"[Drama, Fantasy]",Teenage Matt Banting wants to work with a famo...,Amazon Prime,2021,03,30,min,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9663,s9664,Movie,Pride Of The Bowery,Joseph H. Lewis,"Leo Gorcey, Bobby Jordan",NaN,NaT,1940,7+,60,[Comedy],New York City street principles get an East Si...,Amazon Prime,NaN,NaN,NaN,min,1
9664,s9665,TV Show,Planet Patrol,NaN,"DICK VOSBURGH, RONNIE STEVENS, LIBBY MORRIS, M...",NaN,NaT,2018,13+,4,[TV Shows],"This is Earth, 2100AD - and these are the adve...",Amazon Prime,NaN,NaN,NaN,Seasons,1
9665,s9666,Movie,Outpost,Steve Barker,"Ray Stevenson, Julian Wadham, Richard Brake, M...",NaN,NaT,2008,R,90,[Action],"In war-torn Eastern Europe, a world-weary grou...",Amazon Prime,NaN,NaN,NaN,min,1
9666,s9667,TV Show,Maradona: Blessed Dream,NaN,"Esteban Recagno, Ezequiel Stremiz, Luciano Vit...",NaN,NaT,2021,TV-MA,1,"[Drama, Sports]","The series tells the story of Diego Maradona, ...",Amazon Prime,NaN,NaN,NaN,Season,2


In [38]:
merged_s_df = pd.merge(prime_series_df, 
                     imbd_df[['title', 'original_title', 'director', 'avg_vote', 'production_company', 
                              'duration', 'year', 'genre', 'usa_gross_income', 'worlwide_gross_income']], 
                     on=['title'])
merged_s_df.shape

(136, 26)

In [39]:
merged_s_df

,show_id,type,title,director_x,cast,country,date_added,release_year,rating,duration_x,...,duration_type,original_title,director_y,avg_vote,production_company,duration_y,year,genre_y,usa_gross_income,worlwide_gross_income
0,s116,TV Show,White Dragon,NaN,"John Simm, Anthony Wang, Emilia Fox, Katie Leu...",NaN,NaT,2019,16+,1,...,Season,Fei hap Siu bak lung,Wilson Yip,5.7,China Film Co-Production Corporation,93,2004,"Action, Adventure, Comedy",NaN,$ 646152
1,s138,TV Show,Wendy,NaN,"Christa Clahane, Abbe Hoekstra, Zena Drive, Ky...",NaN,NaT,2016,ALL,1,...,Season,Wendy,Benh Zeitlin,5.7,Fox Searchlight Pictures,111,2020,"Drama, Fantasy",$ 143518,$ 144564
2,s199,TV Show,Vanity Fair,NaN,"Olivia Cooke, Claudia Jessie, Tom Bateman, Joh...",NaN,NaT,2018,13+,1,...,Season,Vanity Fair,Chester M. Franklin,5.7,M.H. Hoffman Inc.,78,1932,"Drama, Romance",NaN,NaN
3,s207,TV Show,Utopia,NaN,"John Cusack, Rainn Wilson, Sasha Lane",United States,NaT,2020,16+,1,...,Season,Utopia,Sohrab Shahid Saless,7.3,Multimedia,198,1983,Drama,NaN,NaN
4,s230,TV Show,Uncle,NaN,"Nick Helm, Elliot Speller-Gillott, Daisy Hagga...",NaN,NaT,2014,TV-14,2,...,Seasons,Uncle,Gireesh Damodar,6.4,Abra Films International,145,2018,Drama,NaN,$ 89724
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,s9355,TV Show,Amnesia,NaN,"Anthony Calf, John Hannah, Jemma Redgrave, Bre...",NaN,NaT,2004,16+,1,...,Season,Amnesia,Gonzalo Justiniano,6.4,Arca Ltda.,90,1994,Drama,NaN,NaN
132,s9355,TV Show,Amnesia,NaN,"Anthony Calf, John Hannah, Jemma Redgrave, Bre...",NaN,NaT,2004,16+,1,...,Season,Amnesia,Kurt Voss,4.6,Dream Entertainment,88,1997,Thriller,NaN,NaN
133,s9355,TV Show,Amnesia,NaN,"Anthony Calf, John Hannah, Jemma Redgrave, Bre...",NaN,NaT,2004,16+,1,...,Season,Amnesia,Nini Bull Robsahm,5.0,Tappeluft Pictures,80,2014,"Drama, Thriller",NaN,NaN
134,s9355,TV Show,Amnesia,NaN,"Anthony Calf, John Hannah, Jemma Redgrave, Bre...",NaN,NaT,2004,16+,1,...,Season,Amnesia,Barbet Schroeder,6.1,Vega Film,96,2015,Drama,$ 8700,$ 8700
